In [ ]:
from IPython import get_ipython
from IPython.display import display

In [ ]:
!pip install tensorflow
!pip install keras
!pip install kaggle
!pip install opencv-python

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import kagglehub
from kagglehub import KaggleDatasetAdapter
from google.colab import files
import os
import pathlib
import cv2
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout

In [ ]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!cp /content/drive/MyDrive/CollabData/kaggle_API/kaggle.json ~/.kaggle/kaggle.json

cp: cannot stat '/content/drive/MyDrive/CollabData/kaggle_API/kaggle.json': No such file or directory


In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
! kaggle datasets download cakrulgaming/indonesia-traffic-sign

Dataset URL: https://www.kaggle.com/datasets/cakrulgaming/indonesia-traffic-sign
License(s): unknown
indonesia-traffic-sign.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
! unzip indonesia-traffic-sign.zip

Archive:  indonesia-traffic-sign.zip
replace traffic_sign/test/lampu-hijau/lampu hijau (100).jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
def detect_lines(image_path):
    """Detects lines in an image using the Hough Line Transform.
    Args:
        image_path: Path to the input image.
    Returns:
        An image with detected lines overlaid.
    """
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)
    lines = cv2.HoughLines(edges, 1, np.pi/180, 200)

    if lines is not None:  # Check if any lines were detected
        for line in lines:
            rho, theta = line[0]
            a = np.cos(theta)
            b = np.sin(theta)
            x0 = a*rho
            y0 = b*rho
            x1 = int(x0 + 1000*(-b))
            y1 = int(y0 + 1000*(a))
            x2 = int(x0 - 1000*(-b))
            y2 = int(y0 - 1000*(a))
            cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 2)

    return img  # Return the image with lines overlaid

In [ ]:
# Create an ImageDataGenerator for preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.2,  # Add this
    height_shift_range=0.2, # Add this
    rotation_range=20,     # Add this
    brightness_range=[0.8, 1.2]  # Add this
)

test_datagen = ImageDataGenerator(rescale=1./255)
early_stopping = EarlyStopping(monitor='val_loss', patience=5)  # Stop if val_loss doesn't improve for 5 epochs

# Load and preprocess the training and testing datasets
train_generator = train_datagen.flow_from_directory(
    '/content/traffic_sign/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    '/content/traffic_sign/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
# Load the pre-trained MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Determine the number of classes (sports ball types) in your dataset
num_classes = 21

# Add a new classification layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(num_classes, activation='softmax', kernel_regularizer=regularizers.l2(0.01))(x)
# Create the final model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=[early_stopping]
)

In [ ]:
# Plot training accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Save the model
model.save('Traffic_Sign_classifier.h5')

In [ ]:
def predict_image(img_path, class_names):
    img = image.load_img(img_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img / 255.

    img_with_lines = detect_lines(img_path) # Add line detection

    # Convert 'img_with_lines' to RGB if needed:
    img_with_lines_rgb = cv2.cvtColor(img_with_lines, cv2.COLOR_BGR2RGB)

    # Continue with your existing prediction process:
    img = image.img_to_array(img_with_lines_rgb)

    prediction = model.predict(img)
    predicted_class_index = np.argmax(prediction)

    # Map predicted_class_index to the corresponding sports ball type
    predicted_class_name = class_names[predicted_class_index]

    return predicted_class_name  # Return the class name

In [ ]:
class_names = list(train_generator.class_indices.keys())

In [ ]:
    uploaded = files.upload()

    for fn in uploaded.keys():
      # predicting images
      path = fn
      img = image.load_img(path, target_size=(224,224))
      predicted_class_name = predict_image(path, class_names) # Call the function here
      print(f"The predicted class for {fn} is: {predicted_class_name}")